In [1]:
import sys
sys.path.append('../')

In [2]:
import psycopg2
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from sqlalchemy import create_engine
from app.utils.config import settings

# Set up connection parameters from environment variables
dbname = settings.DBNAME
user = settings.READ_ONLY_DBUSER
password = settings.READ_ONLY_DBPASS
host = settings.DBHOST
port = settings.DBPORT

# Connect to the PostgreSQL database
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conn_str)

# Load conditions into a dataframe
conditions = pd.read_sql_query('SELECT * FROM cellconditions', engine)

conditions.head()
# load in the conditions of each cell and seperate them
train_conds = conditions[ conditions['dataset'] == 'Train']
test1_conds = conditions[ conditions['dataset'] == 'Prim. Test']
test2_conds = conditions[ conditions['dataset'] == 'Sec. test']

test1_conds = test1_conds.drop(index=42) # Was previously 44 which was incorrect. 

In [3]:
## Extrapolate the capacity fade curves ##

from scipy.optimize import curve_fit

def linear_model(x, a, b):
    return a*x + b

In [4]:
# Get all the x_train data where the folder name is train, test1, test2

all_x = pd.read_sql_query("SELECT x_train FROM celltrainingdatas WHERE folder_name = 'train' OR folder_name = 'test1' OR folder_name = 'test2'", engine)
all_y = pd.read_sql_query("SELECT y_train FROM celltrainingdatas WHERE folder_name = 'train' OR folder_name = 'test1' OR folder_name = 'test2'", engine)


In [5]:

extrapolated_cap_data_all = []
extrapolated_cycle_data_all = []
# Perform extrapolation on the capacity data so the cells last longer. Typically,
# a much lower EOL threshold is used for grid storage.
for i in range(0, len(all_x)):
    print(all_y)
    data = all_y[i]
    x = np.arange(1,len(data)+1,1)
    x = x[-30:]
    y = data[-30:]
    
    extrap_len = 3000
    
    parameters, cov = curve_fit(linear_model, x, y)
    a, b, = parameters
    next_x = np.linspace(x[-1:] + 1, x[-1:]+extrap_len, extrap_len)
    next_y = linear_model(next_x, a, b)
    # idxs = np.where(next_y > 0.790)[0]
    # next_y = next_y[idxs]
    
    ext_data = np.concatenate((data.reshape((-1,1)), next_y), axis=0).reshape(-1)
    extrapolated_cap_data_all.append(ext_data)
    extrapolated_cycle_data_all.append(np.arange(2,len(ext_data)+2,1))

        y_train
0       1.08180
1       1.08320
2       1.08390
3       1.08450
4       1.08480
...         ...
100203  0.83359
100204  0.83170
100205  0.82952
100206  0.82746
100207  0.82577

[100208 rows x 1 columns]


KeyError: 0